# scratch work

In [1]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

import numpy as np
import matplotlib.pyplot as plt

from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.jplhorizons import Horizons

from jorbit import Particle

In [23]:
# a = [1*u.arcsec, 2*u.arcsec, 3*u.arcsec]
a = np.array([1, 2, 3])
np.array([np.diag(np.array([a**2, a**2])) for a in a])

array([[[1, 0],
        [0, 1]],

       [[4, 0],
        [0, 4]],

       [[9, 0],
        [0, 9]]])

In [17]:
a = 1 * u.arcmin
a = np.array(a)
a.shape

()

In [14]:
@jax.jit
def tangent_plane_projection(ra_ref, dec_ref, ra, dec):
    # Convert to unit vectors
    cos_dec = jnp.cos(dec)
    sin_dec = jnp.sin(dec)
    cos_ra = jnp.cos(ra)
    sin_ra = jnp.sin(ra)

    # Initial cartesian coordinates
    x = cos_dec * cos_ra
    y = cos_dec * sin_ra
    z = sin_dec

    # Rotation matrices (combined into single operation)
    cos_ra_ref = jnp.cos(ra_ref)
    sin_ra_ref = jnp.sin(ra_ref)
    cos_dec_ref = jnp.cos(dec_ref)
    sin_dec_ref = jnp.sin(dec_ref)

    # Apply rotations (optimized matrix multiplication)
    x_rot = (x * cos_ra_ref + y * sin_ra_ref) * cos_dec_ref + z * sin_dec_ref
    y_rot = -x * sin_ra_ref + y * cos_ra_ref
    z_rot = -(x * cos_ra_ref + y * sin_ra_ref) * sin_dec_ref + z * cos_dec_ref

    # Project to plane
    xi = y_rot / x_rot
    eta = z_rot / x_rot

    return xi, eta

In [32]:
c1 = SkyCoord(ra=10 * u.deg, dec=89.99 * u.deg, frame="icrs")
c2 = c1.directional_offset_by(position_angle=45 * u.deg, separation=30 * u.arcsec)
ra_ref = c1.ra.rad
dec_ref = c1.dec.rad
ra = c2.ra.rad
dec = c2.dec.rad

xi, eta = tangenet_plane_projection(ra_ref, dec_ref, ra, dec)
np.sqrt(xi**2 + eta**2) * (u.rad.to(u.arcsec)) - 30

np.float64(2.6651654394527213e-07)

In [5]:
import numpyro
from numpyro.distributions import MultivariateNormal, Distribution
import numpyro.distributions as dist


class WeightedSumDistribution(Distribution):
    def __init__(self, weight, base_data):
        super().__init__()
        self.weight = weight
        # base_data defines HOW to compute probabilities
        self.base_data = base_data

    def log_prob(self, value):
        # Here we use both:
        # - self.base_data (from initialization)
        # - value (from the obs argument)
        return -jnp.sum((value - self.weight * self.base_data) ** 2)


def model(data):
    weight = numpyro.sample("weight", dist.Normal(0, 1))

    # First data: tells distribution how to compute probabilities
    # Second data (obs): tells NumPyro what we actually observed
    return numpyro.sample("obs", WeightedSumDistribution(weight, data), obs=data)


model(jnp.array([1.0]))

AssertionError: 